In [3]:
! pip install llama-index == 0.11.13 
! pip install llama-index-llms-ollama
! pip install llama-index-embeddings-ollama

zsh:1: = not found


> Set runtime environment
* connect to local ollama server
* set ollama model name
* put ollama host and model name into environment variables

In [9]:
import os 
import logging
import sys

from llama_index.core import Settings
from llama_index.llms.ollama import Ollama
from llama_index.embeddings.ollama import OllamaEmbedding

ollama_host = os.getenv("OLLAMA_API", "http://localhost:11434")
ollama_model = 'mistral:latest'
logging.basicConfig(level=logging.INFO, stream=sys.stdout)
logger = logging.getLogger(__name__)
logger.info(f"ollama_host: {ollama_host}")
logger.info(f"ollama_model: {ollama_model}")
llm = Ollama(model=ollama_model, base_url=ollama_host, request_timeout=120)
embedding = OllamaEmbedding(model_name=ollama_model, base_url=ollama_host)
Settings.embed_model = embedding
Settings.llm = llm
print(await llm.acomplete(prompt="Hi?"))


INFO:__main__:ollama_host: http://localhost:11434
INFO:__main__:ollama_model: mistral:latest


RemoteProtocolError: Server disconnected without sending a response.

In [6]:
from llama_index.core import SimpleDirectoryReader
documents = SimpleDirectoryReader(input_dir="./ch2/files").load_data()

> Use IngestionPipeline to ingest documents (not working now)

In [ ]:
from llama_index.core.ingestion import IngestionPipeline
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core.extractors import TitleExtractor
import asyncio, nest_asyncio

pipeline = IngestionPipeline(
    transformations = [
        SentenceSplitter(chunk_size=1000, chunk_overlap=200),
        TitleExtractor(),
        embedding
    ]
)
nodes = pipeline.run(documents=documents, num_workers=4)
logger.debug(f"nodes: {len(nodes)}")
for node in nodes:
    logger.debug(f"node: {node}")

> LlamaIndex Support different types of indexes

* SummaryIndex
* DocumentSummaryIndex
* VectorStoreIndex
* TreeIndex
* KeywordTableIndex
* KnowledgeGraphIndex
* ComposableIndex


In [27]:
from llama_index.core import VectorStoreIndex
index = VectorStoreIndex.from_documents(documents)
query_engine = index.as_query_engine(llm=llm)
resp = query_engine.query("summarize each documents in a few sentences")
logger.debug(f"resp: {resp}")


DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: In ancient Rome, the city of Rome itself was th...
DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: Many people consider dogs to be their loyal com...
DEBUG:httpcore.connection:connect_tcp.started host='localhost' port=11434 local_address=None timeout=None socket_options=None
DEBUG:httpcore.connection:connect_tcp.complete return_value=<httpcore._backends.sync.SyncStream object at 0x1714baa50>
DEBUG:httpcore.http11:send_request_headers.started request=<Request [b'POST']>
DEBUG:httpcore.http11:send_request_headers.complete
DEBUG:httpcore.http11:send_request_body.started request=<Request [b'POST']>
DEBUG:httpcore.http11:send_request_body.complete
DEBUG:httpcore.http11:receive_response_headers.started request=<Request [b'POST']>
DEBUG:httpcore.http11:receive_response_headers.complete return_value=(b'HTTP/1.1', 200, b'OK', [(b'Content-Type', b'application/json; charset=utf-8'), (b'Date', b'Wed, 20 Nov 2024 00:53:56 G